In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import json
tweets = []
for line in open('../input/tweets.json', 'r'):
    tweets.append(json.loads(line))

In [ ]:
obama_cnt = 0
romney_cnt = 0
obama_romney_cnt = 0

import pandas as pd
tweets_df = pd.DataFrame(columns=['id','tweet'],index=None)

for i in range(0,len(tweets)):
    obama = False;
    romney = False;
    for word in tweets[i]['text'].split():
#         print(word)
        if word.lower() in ["obama","barack","barackobama","obamabarack"]:
            obama = True
        if word.lower() in["mitt","romney","mittromney","romneymitt"]:
            romney = True
    if obama == True and romney == False:
        data = pd.DataFrame({"id":[tweets[i]['id_str']],"tweet":[tweets[i]['text']]})
        tweets_df = tweets_df.append(data, ignore_index = True)
        obama_cnt += 1
    elif obama == False and romney == True:
        data = pd.DataFrame({"id":[tweets[i]['id_str']],"tweet":[tweets[i]['text']]})
        tweets_df = tweets_df.append(data, ignore_index = True)
        romney_cnt += 1
#     elif obama == True and romney == True:
#         obama_romney_cnt += 1
print(obama_cnt)
print(romney_cnt)
print(obama_romney_cnt)

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix


In [ ]:
import re
processed_tweet = []
for i in range (0,len(tweets_df)):
    if(i % 10000 == 0):
        print("i:",i)
    x = tweets_df.iloc[i]['tweet']
#     tweets_df.iloc[i]['tweet'] = ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
    temp = ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
    processed_tweet.append(temp)
print(processed_tweet)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_word = TfidfVectorizer(sublinear_tf=True,
                                       analyzer='word',
                                       stop_words='english', 
                                       token_pattern=r'\w{1,}', 
                                       norm='l2',
                                       ngram_range=(1,2),
                                       max_features=50000)
mat = tfidf_vectorizer_word.fit_transform(processed_tweet)

In [16]:
mat.shape

(354897, 50000)

In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(mat)

In [18]:
kmeans.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [20]:
from collections import Counter
Counter(kmeans.labels_)

Counter({0: 336086, 1: 18811})

In [21]:
tweets_df['processed_tweet'] = processed_tweet

In [22]:
tweets_df.head()

,id,tweet,processed_tweet
0,246058206651117568,Can we please stop pretending that Obama is a ...,Can we please stop pretending that Obama is a ...
1,246058207460614144,RT @MotherJones: The official GOP response to ...,The official GOP response to Americans being k...
2,246058208463044608,RT @PolarCoug: Obama would never make a good r...,Obama would never make a good running back He ...
3,246058208584671234,RT @Zack_gale: You cant bag on Obama if you on...,gale You cant bag on Obama if you only pay att...
4,246058212116267008,@SethLavin national pressure on both sides to ...,national pressure on both sides to end it will...


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
# text = word_tokenize("And now for something completely different")

tweets_imp_words_df = pd.DataFrame(columns=['id','Adj_Adv_Verb','pos'],index=None)
for i in range (0,len(tweets_df)):
    if(i % 10000 == 0):
        print("i:",i)
    text = word_tokenize(tweets_df.iloc[i]['processed_tweet'])
    pos_tagged_words = nltk.pos_tag(text)
    tempStr = ''
    pos = []
    for i in range(0,len(pos_tagged_words)):
        if pos_tagged_words[i][1] in ['JJ','JJR','JJS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ']:
            tempStr += pos_tagged_words[i][0]+" "
            pos.append(pos_tagged_words[i][1])
    tempData = pd.DataFrame({"id":[tweets_df.iloc[i]['id']],"Adj_Adv_Verb":[tempStr],"pos":[pos]})
    tweets_imp_words_df = tweets_imp_words_df.append(tempData, ignore_index = True)

i: 0
i: 10000
i: 20000
i: 30000
i: 40000
i: 50000
i: 60000
i: 70000
i: 80000
i: 90000
i: 100000
i: 110000
i: 120000
i: 130000
i: 140000
i: 150000
i: 160000
i: 170000
i: 180000
i: 190000
i: 200000
i: 210000
i: 220000
i: 230000
